In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df_5percent=df[0:1563] 


In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,248,6,1,1,1,...,1,1,5,0,5.578346,7.079124,96256,6.322803,1,1
1,144,3,4,65535,184,296,5,1,0,1,...,1,1,4,0,6.742952,4.259479,1581600,7.247902,1,0
2,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.618253,6.104572,2127360,6.756100,0,0
3,144,3,4,65535,184,192,2,1,0,1,...,1,1,1,0,0.000000,0.000000,15944,5.557162,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,5.908894,4.995389,14336,5.649008,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,144,3,4,65535,184,224,4,1,1,1,...,1,0,4,0,6.917484,7.562492,407552,7.158627,0,1
1559,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,4.966202,6.330855,15360,5.742427,0,1
1560,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.370282,4.485058,854152,5.629453,0,0
1561,144,3,4,65535,184,288,4,1,1,1,...,1,0,4,0,5.611377,7.238213,71168,6.858846,1,1


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (754, 69)
class 1: (809, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     248.0               6.0   
1      144.0   3.0        4.0     65535.0  184.0     296.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     256.0               5.0   
3      144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
4      144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
1558   144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
1559   144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
1560   144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
1561   144.0   3.0        4.0     65535.0  184.0     288.0               4.0   
1562   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,1563.000000,1563.000000,1563.00000,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1563.000000,1563.0,...,1563.00000,1563.00000,1563.000000,1563.000000,1563.000000,1563.000000,1.563000e+03,1563.000000,1563.000000,1563.000000
mean,162.467051,2.995521,3.96929,65096.486244,183.559821,222.738324,4.616763,0.989123,0.364683,1.0,...,0.99872,1.31286,3.303903,0.161868,4.940813,2.597918,8.637157e+05,6.388089,0.550224,0.517594
std,932.764654,3.587302,0.34925,5241.964119,19.862542,49.663399,1.831129,0.103755,0.481495,0.0,...,0.03576,1.52613,1.131735,0.368448,2.524012,2.720902,5.691981e+06,1.098180,0.497630,0.499850
min,0.000000,0.000000,0.00000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942896,0.000000,0.000000
25%,144.000000,3.000000,4.00000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.00000,1.00000,3.000000,0.000000,4.070776,0.000000,6.304000e+04,5.742407,0.000000,0.000000
50%,144.000000,3.000000,4.00000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.00000,1.00000,4.000000,0.000000,6.161186,1.591198,1.228800e+05,6.416433,1.000000,1.000000
75%,144.000000,3.000000,4.00000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.00000,2.00000,4.000000,0.000000,6.504980,4.845211,3.187200e+05,7.276143,1.000000,1.000000
max,37008.000000,144.000000,4.00000,65535.000000,332.000000,592.000000,25.000000,1.000000,1.000000,1.0,...,1.00000,20.00000,7.000000,1.000000,7.997171,7.999568,1.111193e+08,7.999997,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      53.75
1.0      46.25
dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(0.02083735, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.21276432, requires_grad=False), tensor(0.0392345, requires_grad=False)], Y =  1
X = [tensor(-0.41570942, requires_grad=False), tensor(-0.19300579, requires_grad=False)], Y = -1
X = [tensor(-0.21260544, requires_grad=False), tensor(0.03942024, requires_grad=False)], Y = -1
X = [tensor(-0.21273619, requires_grad=False), tensor(0.03938724, requires_grad=False)], Y =  1
X = [tensor(-0.21355325, requires_grad=False), tensor(0.03901339, requires_grad=False)], Y = -1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [29]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1133099 | Accuracy: 0.5625000 
Iter:     2 | Cost: 1.5399952 | Accuracy: 0.5625000 
Iter:     3 | Cost: 1.2894017 | Accuracy: 0.5625000 
Iter:     4 | Cost: 1.0761511 | Accuracy: 0.5625000 
Iter:     5 | Cost: 1.0540826 | Accuracy: 0.3812500 
Iter:     6 | Cost: 1.0899125 | Accuracy: 0.4437500 
Iter:     7 | Cost: 1.0918652 | Accuracy: 0.4437500 
Iter:     8 | Cost: 1.0897032 | Accuracy: 0.4375000 
Iter:     9 | Cost: 1.0827877 | Accuracy: 0.4375000 
Iter:    10 | Cost: 1.0356917 | Accuracy: 0.4437500 
New best
Iter:    11 | Cost: 0.9894479 | Accuracy: 0.5687500 
New best
Iter:    12 | Cost: 0.9777231 | Accuracy: 0.5750000 
New best
Iter:    13 | Cost: 0.9856796 | Accuracy: 0.5812500 
Iter:    14 | Cost: 0.9722094 | Accuracy: 0.5812500 
New best
Iter:    15 | Cost: 0.9393365 | Accuracy: 0.6062500 
Iter:    16 | Cost: 0.9755723 | Accuracy: 0.4500000 
Iter:    17 | Cost: 1.1804024 | Accuracy: 0.4437500 
Iter:    18 | Cost: 1.0888966 | Accuracy: 0.4437500 
I

In [30]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [31]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9191382930431962, Accuracy: 52.0%


In [32]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,-1.0,1.0
2,-1.0,1.0
3,-1.0,1.0
4,-1.0,-1.0
5,-1.0,-1.0
6,-1.0,-1.0
7,-1.0,1.0
8,-1.0,1.0
9,-1.0,-1.0


In [33]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.95      0.50      0.65        36
         1.0       0.14      0.75      0.24         4

    accuracy                           0.53        40
   macro avg       0.55      0.62      0.45        40
weighted avg       0.87      0.53      0.61        40

0.14285714285714285
0.75
0.24
0.625
